In [ ]:

# connection au WS
from azureml.core import Workspace
ws = Workspace.from_config()
ws.get_details()

# Execution du script en tant qu'experience
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Create a script config + target compute ressource
script_config = ScriptRunConfig(source_directory=".",
                                script='training_script.py',
                                compute_target = 'jcharley1',
                                environment=sklearn_env) 

# Submit the experiment
experiment = Experiment(workspace=ws, name='test4')
run = experiment.submit(config=script_config)
run.wait_for_completion()


In [ ]:
#Récupération des fichiers de modèle
# List the files generated by the experiment
for file in run.get_file_names():
    print(file)

# Download a named file
run.download_file(name='output/model.pkl', output_file_path='model.pkl')

TRACKING DES MODELES - LOCAL

In [31]:
from azureml.core import Model
model = Model.register(workspace=ws,
                       model_name='rf',
                       model_path='output/model.pkl', # local path
                       description='rf test1',
                       tags={'data-format': 'CSV'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

Registering model rf


Afficher les modèles inscrits

In [34]:
from azureml.core import Model

for model in Model.list(ws):
    
    # Get model name and auto-generated version
    print(model.name, 'version:', model.version)

rf version: 1


ENREGISTREMENT D'UN JEU DE DONNEES

In [38]:
from azureml.core import Dataset

blob_ds = ws.get_default_datastore()
csv_paths = [(blob_ds, 'data/train_dataset.csv')]
tab_ds = Dataset.Tabular.from_delimited_files(path=csv_paths)
tab_ds = tab_ds.register(workspace=ws, name='csv_table')

DatasetValidationError: DatasetValidationError:
	Message: Cannot load any data from the specified path. Make sure the path is accessible and contains data.
ScriptExecutionException was caused by StreamAccessException.
  StreamAccessException was caused by NotFoundException.
    Found no resources for the input provided: '[REDACTED]'
| session_id=d4bdd079-864e-4910-9c92-17e59921ee89
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot load any data from the specified path. Make sure the path is accessible and contains data.\nScriptExecutionException was caused by StreamAccessException.\r\n  StreamAccessException was caused by NotFoundException.\r\n    Found no resources for the input provided: '[REDACTED]'\r\n| session_id=d4bdd079-864e-4910-9c92-17e59921ee89"
    }
}